In [4]:
import sys
sys.path.append('../')

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from dataset.omsignal_dataset import OMSignalDataset
#from dataset.unlabeled_dataset import UnlabeledOMSignalDataset
from torchvision.transforms import Compose

from dataset.preprocessor import Preprocessor
from dataset.segmenter import PairedHeartBeatSegmenter,HeartBeatSegmenter
from dataset.to_float_tensor_transform import ToFloatTensorTransform
from dataset.flip_transform import FlipTransform
from dataset.shift_transform import ShiftTransform
from dataset.gaussian_noise_transform import GaussianNoiseTransform
from dataset.encode_transform import Encode
from autoencoder import AutoEncoder_2l,AutoEncoder_3l,AutoEncoder_3l_v2


import matplotlib.pyplot as plt

from models.multi_task_model import MultiTaskNet,MultiTaskNet_mlp
from models.multi_task_trainer import MultiTaskTrainer

from utils.dataset_utils import generate_transformed_dataset
from utils.ids_conversion import convert_back_ids,convert_ids

from models.multi_task_predictions import multi_task_predict

from utils.file_utils import read_data
from scipy.stats import kendalltau, mode
from sklearn.metrics import accuracy_score

import pandas as pd
import time

ImportError: cannot import name 'AutoEncoder_3l_v2'

In [2]:
TRAIN_UNLABELED_PATH = "/rap/jvb-000-aa/COURS2019/etudiants/data/omsignal/myHeartProject/MILA_UnlabeledData.dat"
TRAIN_LABELED_PATH = "/rap/jvb-000-aa/COURS2019/etudiants/data/omsignal/myHeartProject/MILA_TrainLabeledData.dat"
VALID_LABELED_PATH = "/rap/jvb-000-aa/COURS2019/etudiants/data/omsignal/myHeartProject/MILA_ValidationLabeledData.dat"

SEG_TRAIN_UNLABELED = "/rap/jvb-000-aa/COURS2019/etudiants/submissions/b2pomt3/OMSignal_SegmentedData_Backup_2.dat"
SEG_TRAIN_LABELED = "/rap/jvb-000-aa/COURS2019/etudiants/submissions/b2pomt3/OMSignal_Segmented_TrainLabeledData_encode.dat"
SEG_VALID_LABELED = "/rap/jvb-000-aa/COURS2019/etudiants/submissions/b2pomt3/OMSignal_Segmented_ValidationLabeledData_encode.dat"

SAVE_PATH = "./results/"

In [3]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
encoder = encoder = AutoEncoder_3l_v2(1, hidden_size=[16,32,64], kernel_size=[13,9,5,5,10,13], padding=[6,4,2,2,4,6], stride=[1,1,1,1,1,1])
encoder.load_state_dict(torch.load("/rap/jvb-000-aa/COURS2019/etudiants/submissions/b2pomt3/AutoEncoder_3l_v2_Adam_train1.tar", map_location=DEVICE)["model_state_dict"])


NameError: name 'AutoEncoder_3l_v2' is not defined

In [13]:
origin_dataset_path = TRAIN_LABELED_PATH
destination_dataset_path = SEG_TRAIN_LABELED 
transforms = Compose([Preprocessor(), HeartBeatSegmenter(110)])
generate_transformed_dataset(True,
                                 origin_dataset_path,
                                 destination_dataset_path,
                                 transforms)

origin_dataset_path = VALID_LABELED_PATH
destination_dataset_path = SEG_VALID_LABELED 
transforms = Compose([Preprocessor(), HeartBeatSegmenter(110)])
generate_transformed_dataset(True,
                                 origin_dataset_path,
                                 destination_dataset_path,
                                 transforms)

In [4]:

transforms_train = Compose([ToFloatTensorTransform()])
transforms_valid =  Compose([ToFloatTensorTransform()])

train_labeled_data = OMSignalDataset(True,True,SEG_TRAIN_LABELED  ,transforms_train)
valid_labeled_data = OMSignalDataset(True,True,SEG_VALID_LABELED ,transforms_valid)
# valid_labeled_data = OMSignalDataset(False,True,VALID_LABELED_PATH,transforms_valid)

trainloader = torch.utils.data.DataLoader(train_labeled_data, batch_size=16,
                                          shuffle=True)

validloader = torch.utils.data.DataLoader(valid_labeled_data , batch_size=16,
                                         shuffle=False)

In [5]:
'''
train model
'''
model = MultiTaskNet(224, 1, 35, 30)
Trainer = MultiTaskTrainer(model, 
                           trainloader,
                           validloader,
                           SAVE_PATH, 
                           learning_rate=0.0001, 
                           batch_size=16, 
                           weight_decay=0.01,
                           log=True,
                           seed=111)

Trainer.train(50)

Start training.
Epoch : 0
Start validation.
Epoch train total loss : 12.64430046081543
Epoch valid total loss : 12.259575843811035
Epoch train ID loss : 3.3914217948913574
Epoch valid ID loss : 3.42523455619812
Epoch train PR loss : 14.194805145263672
Epoch valid PR loss : 13.736130714416504
Epoch train RT loss : 31.65864372253418
Epoch valid RT loss : 29.899869918823242
Epoch train RR loss : 3.802366256713867
Epoch valid RR loss : 3.9609386920928955
Accuracy on train data : 20.91072498502097 %
Accuracy on valid data : 9.786802030456853 %
Epoch : 1
Start validation.
Saving model multi_task_net to : ./results/
Epoch train total loss : 4.675775051116943
Epoch valid total loss : 3.818878412246704
Epoch train ID loss : 3.143576145172119
Epoch valid ID loss : 3.619554042816162
Epoch train PR loss : 3.3288893699645996
Epoch valid PR loss : 1.4683201313018799
Epoch train RT loss : 5.199770927429199
Epoch valid RT loss : 2.200698137283325
Epoch train RR loss : 2.275904893875122
Epoch valid RR 

Epoch valid ID loss : 2.452359199523926
Epoch train PR loss : 1.5680145025253296
Epoch valid PR loss : 1.3469622135162354
Epoch train RT loss : 1.4975101947784424
Epoch valid RT loss : 1.4199007749557495
Epoch train RR loss : 1.827244520187378
Epoch valid RR loss : 0.6403887867927551
Accuracy on train data : 64.6295186738566 %
Accuracy on valid data : 20.324873096446698 %
Epoch : 15
Start validation.
Saving model multi_task_net to : ./results/
Epoch train total loss : 1.9274364709854126
Epoch valid total loss : 3.089857578277588
Epoch train ID loss : 1.2055270671844482
Epoch valid ID loss : 2.590156316757202
Epoch train PR loss : 1.5697444677352905
Epoch valid PR loss : 1.4944682121276855
Epoch train RT loss : 1.4424859285354614
Epoch valid RT loss : 2.8452417850494385
Epoch train RR loss : 1.8028438091278076
Epoch valid RR loss : 0.7489508986473083
Accuracy on train data : 59.416816456960255 %
Accuracy on valid data : 21.868020304568528 %
Epoch : 16
Start validation.
Saving model mult

Epoch train RR loss : 1.7116438150405884
Epoch valid RR loss : 0.7307364344596863
Accuracy on train data : 72.17894947074096 %
Accuracy on valid data : 22.456852791878173 %
Epoch : 29
Start validation.
Saving model multi_task_net to : ./results/
Epoch train total loss : 1.5595393180847168
Epoch valid total loss : 2.494300365447998
Epoch train ID loss : 0.8842074275016785
Epoch valid ID loss : 2.2259702682495117
Epoch train PR loss : 1.3709861040115356
Epoch valid PR loss : 1.2326228618621826
Epoch train RT loss : 1.1901146173477173
Epoch valid RT loss : 1.011062741279602
Epoch train RR loss : 1.6997641324996948
Epoch valid RR loss : 1.32393479347229
Accuracy on train data : 75.0349510685041 %
Accuracy on valid data : 23.675126903553302 %
Epoch : 30
Start validation.
Saving model multi_task_net to : ./results/
Epoch train total loss : 1.5409456491470337
Epoch valid total loss : 2.2085537910461426
Epoch train ID loss : 0.8669649362564087
Epoch valid ID loss : 2.0844082832336426
Epoch tra

Epoch train ID loss : 0.7238929271697998
Epoch valid ID loss : 2.0171680450439453
Epoch train PR loss : 1.2532075643539429
Epoch valid PR loss : 0.6162227988243103
Epoch train RT loss : 1.0956451892852783
Epoch valid RT loss : 1.1263866424560547
Epoch train RR loss : 1.6316938400268555
Epoch valid RR loss : 1.157394528388977
Accuracy on train data : 74.59556620730977 %
Accuracy on valid data : 21.238578680203048 %
Epoch : 44
Start validation.
Saving model multi_task_net to : ./results/
Epoch train total loss : 1.3465754985809326
Epoch valid total loss : 2.838484287261963
Epoch train ID loss : 0.7055717706680298
Epoch valid ID loss : 2.6939473152160645
Epoch train PR loss : 1.2264742851257324
Epoch valid PR loss : 0.7875270247459412
Epoch train RT loss : 1.073644995689392
Epoch valid RT loss : 1.254288911819458
Epoch train RR loss : 1.6104713678359985
Epoch valid RR loss : 1.374815821647644
Accuracy on train data : 80.12782105052926 %
Accuracy on valid data : 22.923857868020306 %
Epoch 

(24.690355329949238, 0.3019215166568756)

<Figure size 432x288 with 0 Axes>

In [6]:
'''
predict

'''

model = MultiTaskNet(224, 1, 35, 30)
model.load_state_dict(torch.load('./results/multi_task_net.pt'))

test_transforms = Compose([Preprocessor(),
                           PairedHeartBeatSegmenter(230)
                           ])

test_data = OMSignalDataset(False, True, VALID_LABELED_PATH, test_transforms)

y_pred,true_labels = multi_task_predict(model,test_data,Encode(encoder))

In [7]:
'''
print Metrics

'''

converted_ids = convert_ids(true_labels[:,3])

print("ID: " + str(accuracy_score(converted_ids,y_pred[:,3])))
print("PR: " + str(kendalltau(true_labels[:,0],y_pred[:,0])[0]))
print("RT: " + str(kendalltau(true_labels[:,1],y_pred[:,1])[0]))
print("RR: " + str(kendalltau(true_labels[:,2],y_pred[:,2])[0]))

ID: 0.30625
PR: 0.44519579100568857
RT: 0.8089322320498982
RR: 0.2931719016377315
